<a href="https://colab.research.google.com/github/pachterlab/scATAK/blob/main/scATAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import time
start_time = time.time()

In [5]:
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R1.fastq.gz?raw=true > min-R1.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R2.fastq.gz?raw=true > min-R2.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R3.fastq.gz?raw=true > min-R3.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/lib/737K-cratac-v1.txt?raw=true > 737K-cratac-v1.txt

In [6]:
# kb python for quantification
!pip install --quiet kb-python

# install minimap for mapping paired end reads
!wget --quiet --show-progress "https://github.com/lh3/minimap2/releases/download/v2.24/minimap2-2.24_x64-linux.tar.bz2"
!tar -jxvf minimap2-2.24_x64-linux.tar.bz2
!cp minimap2-2.24_x64-linux/minimap2 /usr/local/bin

# Insstall sambamba for sam file manipulation
!wget --quiet --show-progress https://github.com/biod/sambamba/releases/download/v0.8.2/sambamba-0.8.2-linux-amd64-static.gz
!gunzip sambamba-0.8.2-linux-amd64-static.gz
!chmod +x /content/sambamba-0.8.2-linux-amd64-static
!mv sambamba-0.8.2-linux-amd64-static /usr/local/bin/sambamba

# Genrich for peak callling
!git clone https://github.com/jsh58/Genrich.git
!cd Genrich && make && cp Genrich /usr/local/bin

# bedtools for slicing the peaks
!wget --quiet --show-progress https://github.com/arq5x/bedtools2/releases/download/v2.30.0/bedtools.static.binary
!chmod +x /content/bedtools.static.binary
!mv /content/bedtools.static.binary /usr/local/bin/bedtools

!git clone --quiet https://github.com/pachterlab/kallisto && cd kallisto && git checkout shareseq
!apt-get -qq install autoconf
!cd kallisto && mkdir build && cd build && cmake .. && make &> /dev/null
!cp kallisto/build/src/kallisto $(pip show kb-python | grep "Location" | cut -f2 -d":")/kb_python/bins/linux/kallisto/kallisto
!cp kallisto/build/src/kallisto /usr/local/bin
!cp $(pip show kb-python | grep "Location" | cut -f2 -d":")/kb_python/bins/linux/bustools/bustools /usr/local/bin

     |████████████████████████████████| 7.5 MB 5.8 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 4.8 MB 51.0 MB/s 
     |████████████████████████████████| 45.2 MB 56 kB/s 
     |████████████████████████████████| 2.0 MB 42.6 MB/s 
     |████████████████████████████████| 15.1 MB 22.1 MB/s 
     |████████████████████████████████| 11.2 MB 22.2 MB/s 
     |████████████████████████████████| 88 kB 7.5 MB/s 
     |████████████████████████████████| 960 kB 64.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
minimap2-2.24_x64-l 100%[===================>]   2.04M  --.-KB/s    in 0.1s    
minimap2-2.24_x64-linux/
minimap2-2.24_x64-linux/LICENSE.txt
minimap2-2.24_x64-linux/k8
minimap2-2.24_x64-linux/README-js.md
minimap2-2.24_x64-linux/paftools.js
minimap2-2.24_x64-linux/minimap2
minimap2-2.24_x64-linux/README.md
minimap2-2.24_x64-linux/cookbook.md
minimap2-2.24_x64-linux/

In [7]:
!kallisto bus --list

List of supported single-cell technologies

short name       description
----------       -----------
10xv1            10x version 1 chemistry
10xv2            10x version 2 chemistry
10xv3            10x version 3 chemistry
10xATAC          10x ATAC
Bulk             Bulk RNA-seq or Smart-seq2 (multiplexed)
BDWTA            BD Rhapsody WTA
CELSeq           CEL-Seq
CELSeq2          CEL-Seq version 2
DropSeq          DropSeq
inDropsv1        inDrops version 1 chemistry
inDropsv2        inDrops version 2 chemistry
inDropsv3        inDrops version 3 chemistry
SCRBSeq          SCRB-Seq
SHARE-seq        SHARE-seq RNA
SmartSeq3        Smart-seq3
SPLiT-seq        SPLiT-seq
SureCell         SureCell for ddSEQ
Visium           10x Visium Spatial Transcriptomics



In [8]:
!minimap2

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM[,NUM] chaining/alignment bandwidth and long-join bandwidth [500,20000]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all m

In [9]:
!sambamba


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)


Usage: sambamba [command] [args...]

  Available commands:

    view        view contents and convert from one format
                to another (SAM/BAM/JSON/UNPACK)
    index       build index (BAI)
    merge       merge files (BAM)
    sort        sort file (BAM)
    slice       slice file (BAM using BED)
    markdup     mark or remove duplicates (BAM)
    subsample   subsample (BAM)
    flagstat    output statistics (BAM)
    depth       output statistics (BAM)
    validate    simple validator (BAM)

  No longer recommended:

    mpileup     parallel execution of samtools (BAM)

To get help on a particular command, call it without args.

Global options

    -q          quiet mode (do not show banner)

For bug reports and feature requests see

       https://github.com/biod/



In [10]:
!Genrich

Error! Need input/output files
Usage: ./Genrich  -t <file>  -o <file>  [optional arguments]
Required arguments:
  -t  <file>       Input SAM/BAM file(s) for experimental sample(s)
  -o  <file>       Output peak file (in ENCODE narrowPeak format)
Optional I/O arguments:
  -c  <file>       Input SAM/BAM file(s) for control sample(s)
  -f  <file>       Output bedgraph-ish file for p/q values
  -k  <file>       Output bedgraph-ish file for pileups and p-values
  -b  <file>       Output BED file for reads/fragments/intervals
  -R  <file>       Output file for PCR duplicates (only with -r)
Filtering options:
  -r               Remove PCR duplicates
  -e  <arg>        Comma-separated list of chromosomes to exclude
  -E  <file>       Input BED file(s) of genomic regions to exclude
  -m  <int>        Minimum MAPQ to keep an alignment (def. 0)
  -s  <float>      Keep sec alns with AS >= bestAS - <float> (def. 0)
  -y               Keep unpaired alignments (def. false)
  -w  <int>        Keep unp

In [11]:
!bedtools

bedtools is a powerful toolset for genome arithmetic.

Version:   v2.30.0
About:     developed in the quinlanlab.org and by many contributors worldwide.
Docs:      http://bedtools.readthedocs.io/
Code:      https://github.com/arq5x/bedtools2
Mail:      https://groups.google.com/forum/#!forum/bedtools-discuss

Usage:     bedtools <subcommand> [options]

The bedtools sub-commands include:

[ Genome arithmetic ]
    intersect     Find overlapping intervals in various ways.
    window        Find overlapping intervals within a window around an interval.
    closest       Find the closest, potentially non-overlapping interval.
    coverage      Compute the coverage over defined intervals.
    map           Apply a function to a column for each overlapping interval.
    genomecov     Compute the coverage over an entire genome.
    merge         Combine overlapping/nearby intervals into a single interval.
    cluster       Cluster (but don't merge) overlapping/nearby intervals.
    complement

In [12]:
!wget --quiet --show-progress "http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz"

Homo_sapiens.GRCh38 100%[===================>] 840.39M  20.6MB/s    in 41s     


In [13]:
!time minimap2 \
-d ref.mmi \
Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

tcmalloc: large alloc 1073741824 bytes == 0x69590000 @  0x7f624e09a2a4 0x414d2d 0x406c48 0x4129cf 0x7f624d8936db 0x7f624d5bc61f
tcmalloc: large alloc 2147483648 bytes == 0x27c916000 @  0x7f624e09a2a4 0x406fee 0x4129cf 0x7f624d8936db 0x7f624d5bc61f
[M::mm_idx_gen::114.167*1.62] collected minimizers
[M::mm_idx_gen::147.115*1.70] sorted minimizers
[M::main::180.251*1.52] loaded/built the index for 194 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::182.304*1.52] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -d ref.mmi Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
[M::main] Real time: 184.058 sec; CPU: 278.006 sec; Peak RSS: 11.297 GB

real	3m4.686s
user	3m58.890s
sys	0m39.731s


In [14]:
!time minimap2 -ax sr -t2 ref.mmi \
   <(paste \
     <(zcat min-R1.fastq.gz) \
     <(zcat min-R2.fastq.gz) \
     | awk -v blen="16" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   <(paste \
     <(zcat min-R3.fastq.gz) \
     <(zcat min-R2.fastq.gz) \
     | awk -v blen="16" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   > peaks.sam

tcmalloc: large alloc 1549877248 bytes == 0x17e694000 @  0x7feda71bf1e7 0x40a494 0x40a9b8 0x402995 0x7feda65e2c87 0x4018b9
[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::31.118*0.45] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::31.118*0.45] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::32.777*0.48] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::worker_pipeline::75.276*1.25] mapped 500000 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax sr -t2 ref.mmi /dev/fd/63 /dev/fd/62
[M::main] Real time: 76.279 sec; CPU: 95.168 sec; Peak RSS: 7.617 GB

real	1m16.805s
user	1m26.515s
sys	0m11.598s


In [15]:
!sambamba view -t 2 -f bam -S -o peaks.bam peaks.sam
!rm peaks.sam


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)



In [16]:
!time sambamba sort -n -t 2 -m 8GB --tmpdir=./ peaks.bam


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)


real	0m3.492s
user	0m6.423s
sys	0m0.244s


In [17]:
!time Genrich -t peaks.sorted.bam -o peaks.narrowPeak -f peaks.log -v

Processing experimental file #0: peaks.sorted.bam
  BAM records analyzed:      500166
    Unmapped:                 25581
    Supp./dups/lowQual:         166
    Paired alignments:       447022
    Unpaired alignments:      27397
  Fragments analyzed:        223511
    Full fragments:          223511
      (avg. length: 174.4bp)
- control file #0 not provided -
  Background pileup value: 0.012576
Peak-calling parameters:
  Genome length: 3099750718bp
  Significance threshold: -log(p) > 2.000
  Min. AUC: 200.000
  Max. gap between sites: 100bp
Peaks identified: 150592 (32552110bp)

real	0m22.507s
user	0m16.206s
sys	0m6.115s


In [18]:
!cat peaks.narrowPeak | bedtools sort | bedtools merge > atac_all_peaks.bed

In [19]:
!gunzip Homo_sapiens.GRCh38.dna.primary_assembly.fa \
&& fold -w 80 Homo_sapiens.GRCh38.dna.primary_assembly.fa > genome.fa

In [20]:
!bedtools getfasta -fi genome.fa -bed atac_all_peaks.bed -fo atac_all_peaks.fa

index file genome.fa.fai not found, generating...


In [21]:
!ls -lht atac_all_peaks.fa

-rw-r--r-- 1 root root 35M Oct  7 23:49 atac_all_peaks.fa


In [22]:
!time kallisto index \
-i peaks.idx \
atac_all_peaks.fa


[build] loading fasta file atac_all_peaks.fa
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 2 target sequences
[build] warning: replaced 559 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 329802 contigs and contains 26747917 k-mers 


real	1m44.778s
user	1m42.205s
sys	0m1.879s


In [23]:
!ls -lht peaks.idx

-rw-r--r-- 1 root root 572M Oct  7 23:50 peaks.idx


In [24]:
!cat atac_all_peaks.fa | awk '{if($1~/>/)print substr($1, 2)"\t"substr($1, 2)"\t"substr($1, 2)}' > t2g.txt

In [25]:
!head t2g.txt

1:10015-10460	1:10015-10460	1:10015-10460
1:11154-11266	1:11154-11266	1:11154-11266
1:29210-29373	1:29210-29373	1:29210-29373
1:29480-29798	1:29480-29798	1:29480-29798
1:99502-99868	1:99502-99868	1:99502-99868
1:104185-104255	1:104185-104255	1:104185-104255
1:143396-143517	1:143396-143517	1:143396-143517
1:149332-149519	1:149332-149519	1:149332-149519
1:180546-180929	1:180546-180929	1:180546-180929
1:181290-181834	1:181290-181834	1:181290-181834


In [26]:
!zcat min-R1.fastq.gz | head -2
!zcat min-R2.fastq.gz | head -2
!zcat min-R3.fastq.gz | head -2

@A00836:480:HJ2W5DSXY:1:1101:30355:1016 1:N:0:CCGAGAAC
CNCCGGACCCACTCCCATTACCGTCTGGATCAGCAGCTCCAGGTCGCGGG
@A00836:480:HJ2W5DSXY:1:1101:30355:1016 2:N:0:CCGAGAAC
CCGTAAACACCAAAGC
@A00836:480:HJ2W5DSXY:1:1101:30355:1016 3:N:0:CCGAGAAC
CCCTAGTGCTGTCCGTGACTTCTTTGGATCATGTCTGGCCCAAGATGGG


In [27]:
!kb count \
-i peaks.idx \
-g t2g.txt \
-x 10xATAC \
-o out \
-w 737K-cratac-v1.txt \
-t 2 \
--h5ad \
min-R2.fastq.gz min-R1.fastq.gz min-R3.fastq.gz

[2022-10-07 23:50:56,040]    INFO [count] Using index peaks.idx to generate BUS file to out from
[2022-10-07 23:50:56,040]    INFO [count]         min-R2.fastq.gz
[2022-10-07 23:50:56,041]    INFO [count]         min-R1.fastq.gz
[2022-10-07 23:50:56,041]    INFO [count]         min-R3.fastq.gz
[2022-10-07 23:51:02,971]    INFO [count] Sorting BUS file out/output.bus to out/tmp/output.s.bus
[2022-10-07 23:51:08,397]    INFO [count] Inspecting BUS file out/tmp/output.s.bus
[2022-10-07 23:51:10,008]    INFO [count] Correcting BUS records in out/tmp/output.s.bus to out/tmp/output.s.c.bus with whitelist 737K-cratac-v1.txt
[2022-10-07 23:51:13,032]    INFO [count] Sorting BUS file out/tmp/output.s.c.bus to out/output.unfiltered.bus
[2022-10-07 23:51:18,060]    INFO [count] Generating count matrix out/counts_unfiltered/cells_x_genes from BUS file out/output.unfiltered.bus
[2022-10-07 23:51:20,075]    INFO [count] Reading matrix out/counts_unfiltered/cells_x_genes.mtx
[2022-10-07 23:51:21,075]

In [28]:
# Since we are doing read counting we need to use `--cm`, once this is implemented in kb we can just do it there
!bustools count \
-o out/counts_unfiltered/cells_x_genes \
-g t2g.txt \
-e out/matrix.ec \
-t out/transcripts.txt \
--genecounts \
--cm \
out/output.unfiltered.bus

In [29]:
(time.time()-start_time)/60

11.038964438438416